# Youtbe Video Category Prediciton

In [1]:
import numpy as np
import scipy
import pandas as pd
import xgboost as xgb
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report

In [2]:
final = pd.read_csv("Final.csv") 

In [3]:
final.drop(['Unnamed: 0', 'link'], axis=1,inplace = True)

In [4]:
final.dropna(inplace=True)
final.head()

,title,description,category
0,Ep 1| Travelling through North East India | Of...,"The journey to Arunachal, North East India beg...",Travel Blogs
1,Backpacking In Meghalaya | NorthEast India Tri...,"In this video I explored North East India, sta...",Travel Blogs
2,10 Best Places to Visit in Austria - Travel Video,Austria is a country that is as well known for...,Travel Blogs
3,Salzburg Vacation Travel Guide | Expedia,https://www.expedia.com/Salzburg.d180...\n\nTh...,Travel Blogs
4,Vienna Vacation Travel Guide | Expedia,https://www.expedia.com/Vienna.d17831...\n\nOn...,Travel Blogs


# Data Preprocessing

In [5]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopwords = stopwords.words("english")
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lmt = WordNetLemmatizer()

In [6]:
final['title'] = final['title'].apply(str)
final['description'] = final['description'].apply(str)

In [7]:
from sklearn.utils import shuffle
final = shuffle(final)

In [8]:
#x_train, x_test, y_train, y_test = train_test_split(final[['title', 'description']],final[['category']], test_size=0.2)

In [9]:
#Tokkenizing
title= []
for i in range(len(final)):
    required = (word_tokenize(final['title'].iloc[i]))
    title.append(required)

In [10]:
description= []
for i in range(len(final)):
    required = (word_tokenize(final['description'].iloc[i]))
    description.append(required)

In [11]:
len(title)==len(description)

True

In [12]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [13]:
def clean_data(words):
    after_clean_words = []
    for word in words:
        if word.lower() not in stopwords:
            pos = pos_tag([word])
            clean_word = lmt.lemmatize(word,pos=get_simple_pos(pos[0][1]))
            after_clean_words.append(clean_word)
    return after_clean_words 

In [14]:
stopwords.append("like")
stopwords.append("subscribe")
stopwords.append("comment")
stopwords.append("?")
stopwords.append("|")
stopwords.append(":")
stopwords = stopwords + list(string.punctuation)

In [15]:
final_title = [(clean_data(t)) for t in title]

In [16]:
final_description = [(clean_data(d)) for d in description]

In [17]:
final['title']=final_title
final['description']=final_description

In [18]:
final.head()

,title,description,category
2677,"[Purpose, concept, art, music]","[short, film, concept, Alex, Bourne, Wilkie]",Art & Music
2260,"[HISTORY, IDEAS, Romanticism]","[Romanticism, historical, movement, still, hug...",History
1293,"[EPIC, EID, FEAST, Oman, Middle, Eastern, Trad...","[🎥UNSEEN, Desert, Food, Oman, ’, Bedouin, Peop...",Food
251,"[Taipei, Taiwan, Food, Travel, Blog, 1, Din, T...","[www.imalwayseating.com, instagram, taesungsay...",Travel Blogs
1990,"[10, Shortest, Matches, WWE, History]","[top, 10, quick, Wrestling, match, history, WW...",History


In [19]:
final['title']=final['title'].apply(', '.join)
final['description']=final['description'].apply(', '.join)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
X = CountVectorizer(max_features = 3000,tokenizer=lambda doc: doc, lowercase=False).fit_transform(final_title,final_description)

In [21]:
category=final['category']

In [22]:
Y = final.iloc[:, 2].values

In [23]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, final.iloc[:, 2], test_size = 0.20, random_state = 0)

# Predictions

# Naive Bayes

In [24]:
from sklearn import model_selection, naive_bayes
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [25]:
y_pred= Naive.predict(x_test)

In [34]:
print(Naive.score(x_test, y_test)*100)

97.92746113989638


In [27]:
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

  Art & Music       0.97      0.99      0.98       102
         Food       0.97      0.97      0.97        92
      History       0.97      0.98      0.97        92
Manufacturing       1.00      1.00      1.00        58
      Science       0.97      0.97      0.97        99
 Travel Blogs       1.00      0.98      0.99       136

     accuracy                           0.98       579
    macro avg       0.98      0.98      0.98       579
 weighted avg       0.98      0.98      0.98       579



## SVM

In [29]:
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train, y_train)

pred_SVM = SVM.predict(x_test)

print("Accuracy= ",accuracy_score(y_test,pred_SVM)*100)

Accuracy=  97.23661485319516


In [30]:
print(classification_report(y_test, pred_SVM))

               precision    recall  f1-score   support

  Art & Music       1.00      1.00      1.00       102
         Food       0.99      0.97      0.98        92
      History       0.98      1.00      0.99        92
Manufacturing       1.00      0.93      0.96        58
      Science       0.95      0.94      0.94        99
 Travel Blogs       0.94      0.98      0.96       136

     accuracy                           0.97       579
    macro avg       0.98      0.97      0.97       579
 weighted avg       0.97      0.97      0.97       579



# XGBoost

In [31]:
from sklearn.metrics import f1_score, classification_report, accuracy_score
xgb_model = xgb.XGBClassifier(max_depth=55, n_estimators=100, learning_rate=0.1, colsample_bytree=.7, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(x_train, y_train) 
xgb_prediction = xgb_model.predict(x_test)

In [35]:
accuracy = accuracy_score(y_test, xgb_prediction)
print(accuracy*100)

96.71848013816926


In [33]:
print(classification_report(y_test, xgb_prediction))

               precision    recall  f1-score   support

  Art & Music       0.99      0.99      0.99       102
         Food       0.97      0.97      0.97        92
      History       0.98      0.98      0.98        92
Manufacturing       1.00      0.93      0.96        58
      Science       0.92      0.98      0.95        99
 Travel Blogs       0.97      0.95      0.96       136

     accuracy                           0.97       579
    macro avg       0.97      0.97      0.97       579
 weighted avg       0.97      0.97      0.97       579

